# Titanic Top 4% with ensemble modeling

* **1 소개**
* **2 데이터 로드 및 체크**
    * 2.1 데이터 로드 
    * 2.2 이상치 감지
    * 2.3 학습 및 테스트 데이터 세트 조인
    * 2.4 널 및 결측치 
* **3 피처 분석**
    * 3.1 수치형 값
    * 3.2 범주형 값
* **4 결측치 처리**
    * 4.1 Age
* **5 피처 엔지니어링**
    * 5.1 Name/Title
    * 5.2 Family Size
    * 5.3 Cabin
    * 5.4 Ticket
* **6 모델링**
    * 6.1 단순 ML모델
        * 6.1.1 교차 검증 모델
        * 6.1.2 베스트모델에 대한 하이터 파라미터 튜닝
        * 6.1.3 학습곡선 폴롯
        * 6.1.4 트리기반의 분류기에 대한 피처 중요도
    * 6.2 앙상블 모델링
        * 6.2.1 모델 조합
    * 6.3 예측
        * 6.3.1 예측 및 결과 예측
    

## 1. 소개

캐글에서 세번째로 타이타닉 관련 데이터셋을 가지고  다음과 같은 방법으로 진행하려고 한다.

No matter what you're going through, there's a light at the end of the tunnel. 

It may seem hard to get to it, but you can do it.

Just keep working towards to it and you will find the positive side of things.
  [Demi Lovato]

* **피처 분석**
* **피처 엔지니어링**
* **모델링**

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from collections import Counter
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, ExtraTreesClassifier, VotingClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis  # LDA
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV, cross_val_score, StratifiedKFold, learning_curve, train_test_split

sns.set(style='white', context='notebook', palette='deep', font_scale=1)
plt.style.use('seaborn')

## 2.데이터 로드 및 데이터 확인
### 2.1 데이터 로드 

In [ ]:
def load_dataset():
    train = pd.read_csv("../input/titanic/train.csv")
    test  = pd.read_csv("../input/titanic/test.csv")
    IDTEST = test["PassengerId"]  # 결과물제출시 PassengerId
    
    # Shape및 데이터보기
    display(train.shape, test.shape)
    display(train.head(n=2), display(test.tail(n=2)))
    
    train_len = train.shape[0]
    display("train_len : ", train_len)
    return train, test, IDTEST, train_len

# 수행
train, test, IDTEST, train_len = load_dataset()

### 2.2 이상치 감지

In [ ]:
def detect_outliers(df, features, n):
    """
    df - DataFrame
    features - feature
    n - 이상치값 필터링에 사용되는 변수값
    """
    outlier_index = []
    
    for col in features:
        
        # 25%
        Q1 = np.percentile(df[col], 25)
        Q3 = np.percentile(df[col], 75)
        
        # IQR(Interquartile range)
        IQR = Q3 - Q1
        
        # Outline_step
        outlier_step = 1.5 * IQR
        outlier_df = df[(df[col] < Q1 - outlier_step) | (df[col] > Q3 + outlier_step)]
        display("이상치 : \n", outlier_df, outlier_df.shape)
        
        outlier_list_col = outlier_df.index
        outlier_index.extend(outlier_list_col)
        
    outlier_index = Counter(outlier_index)
    multiple_outliers = list(k for k, v in outlier_index.items() if v > n)
    
    return multiple_outliers

# 이상치탐지
multiple_outliers = detect_outliers(train, ['Age', 'SibSp', 'Parch', 'Fare'], 2)

이상치(Outlier)는 데이터의 통계치를 왜곡시키기 때문에 때론, 아웃라이어를 통계치에 제외시키기도 한다.

1977년도 Turjey, JW가 발표한 Turkey Method에 방법에 의해 아웃라이어를 찾기위해 IQR에 outlier step만큼 더하거나 뺀 값이 이상치이다.

In [ ]:
def remove_outliers(df, outlier_pos):
    """
    이상치의 데이터프레임에서 인덱스를 매개변수로 받아 
    해당 데이터를 삭제
    """
    display("이상치 삭제전 : ", df.shape)
    
    # show he outlier rows
    display(df.iloc[outlier_pos])
    
    # Drop outlier 
    df = df.drop(outlier_pos, axis=0, errors='ignore').reset_index(drop=True)
    # 삭제후 
    display("이상치 삭제 후 ", df.shape)
    return df

# Remove outlier
# 10개의 아웃라이어
train = remove_outliers(train, multiple_outliers)

In [ ]:
train.info()

10개의 이상치 데이터가 발견되었으면 , 28번, 89번 그리고 342번의 승객의 요금이 비정상적으로 높음.

나머지 7개 로우는 매우 높은 SibSp값을 갖고 있다.

### 2.3 학습 데이터셋트와 테스트 데이터세트 조인

In [ ]:
def get_join_dataset(train, test):
    """
    학습 및 테스트 데이터셋을 병합한 데이터셋 
    """
    dataset = pd.concat(objs=[train, test], axis = 0).reset_index(drop=True)
    
    display("\n 병합된 데이터셋 ", dataset.shape)
    
    display("\n Dataset Info ", dataset.info())
    return dataset

# 수행
dataset = get_join_dataset(train, test)

### 2.4 널값과 결측치 확인

In [ ]:
def check_null_missing_val(df, datasetT=True):
    """
    널값과 결측치 확인
    trainT 파라미터는 True인 경우는 학습데이터 확인용
    """
    if datasetT:
        # Fill empty and NaNs values with NaN
        df = df.fillna(np.nan)
    
    # Check for Null values
    display(df.isnull().sum())
    
    # 데이터 타입
    display(df.dtypes, type(df.dtypes), df.dtypes.to_list())
    
    print("\n")
    display(df.info())
    
    print("\n Technical Statistics")
    display(df.describe())
    
    print("\n Top 5")
    display(df.head(n=5))
    
    return df

In [ ]:
dataset = check_null_missing_val(dataset, datasetT=True)

In [ ]:
# train데이터셋 확인
train = check_null_missing_val(train)

Age및 Cabin피처는 결측치 처리를 위해서 중요한 역할을 할 수 있다.

## 3. 피처 분석
### 3.1 숫자형 값

In [ ]:
int_cols = train.dtypes[train.dtypes != 'object']
int_cols.drop(index = ['PassengerId', 'Pclass'], inplace=True)

In [ ]:
def extract_num_feature(df, objType=None):
    """
    데이터프레임을 입력받아 데이터타입을 필터링하는 해당 
    피처만 리스트타입으로 리턴
    리턴값은 pd.Series
    """
    int_cols = df.dtypes[df.dtypes != objType]
    int_cols.drop(index = ['PassengerId', 'Pclass'], inplace=True)
    
    return int_cols  

In [ ]:
int_cols = extract_num_feature(train, 'object')
int_cols.index

In [ ]:
# Correlation matrix between numerical values (SibSp Parch Age and Fare values) and Survived 
g = sns.heatmap(train[int_cols.index].corr(), annot=True, fmt=".2f", cmap = "coolwarm_r")

Fare피처가 생존률과는 의미있는 상관관계를 가지고있습니다. 

그렇다고 해서 다른 피처들이 전혀 필요없다는 의미는 아닙니다. 기존 피처를 이용해 새롭게 파생변수를 생성해보면 상관관계를 새롭게 도출되는 경우도 있습니다. 그런면에서 다른 피처들은 또다른 의미를 갖습니다.

"호랑이는 죽어서 가죽을 남기고 사람은 죽어서 이름을 남긴다."
피처의 소멸 자체도 의미있다는 말씀이지요.

#### SibSP
* [seaborn - factorplot](https://www.kite.com/python/docs/seaborn.factorplot)
* [seaborn - style 사용자 정의](http://hleecaster.com/python-seaborn-set-style-and-context/)

In [ ]:
g = sns.factorplot(x='SibSp', y='Survived', data = train, kind='bar', size=6, palette='muted')
g.set_xlabels("SibSp")
g.despine(left=True)
g.set_ylabels("Survival Probability")

동반가족이 2명보다 많은 경우는 생존률이 많이 낮아짐을 알수 있다.
피처 엔지니어링을 통해서 새로운 피처를 추가적으로 도출이 가능하다.

#### Parch

In [ ]:
g = sns.factorplot(x='Parch', y='Survived', data=train, palette='summer_r', kind='bar')
g.despine(left=True)
g.set_xlabels("Parch")
g.set_ylabels("Survival Possibility")

사이즈가 작은 이동이 생존률이 훨씬 좋은 것을 알수있다. 

동반가족이 3인인 경우, 생존율의 표준편차가 심한 것을 알수 있다.

#### Age

In [ ]:
# 히스토그램과 kde plot이 같이.
g = sns.FacetGrid(train, col='Survived')
g = g.map(sns.distplot, "Age")

연령대를 구분할 필요가 있어보임.연령에 대한 사망과 생존의 차이는 크게 두드러지지는 않지만, 연령대가 뒤로 갈수록 사망과 생존의 구별은 확연히 드러날것으로 보임

In [ ]:
cond_0 = (train["Survived"] == 0 ) & (train["Age"].notnull())  # Not Survived
cond_1 = (train["Survived"] == 1 ) & (train["Age"].notnull()) # Survived

g = sns.kdeplot(train["Age"][cond_0], color="red", shade=True)
g = sns.kdeplot(train["Age"][cond_1], color="Blue", shade=True, ax= g)
g.set_xlabel("Age")
g.set_ylabel("Frequency")
g = g.legend(["Not Survived", "Survived"])

When we superimpose the two densities , we cleary see a peak correponsing (between 0 and 5) to babies and very young childrens.

#### Fare

In [ ]:
def show_dist_plot(df, col:str):
    """
    해당 데이터프레임의 피처에 분포 및 kde plot을 시각화
    """
    g = sns.distplot(a=df[col], color="b", label="skewness : {:.2f}".format(df[col].skew()))
    g.set_title("Skewnewss for {0}".format(col))
    g = g.legend(loc="best")

In [ ]:
num_cols = int_cols[1:].index.values
display(num_cols)

# 학습 및 테스트 데이터셋 조인한 데이터셋
dataset[num_cols].isnull().sum()

In [ ]:
#Fill Fare missing values with the median value
dataset["Fare"] = dataset["Fare"].fillna(dataset["Fare"].median())

In [ ]:
# Recheck 
dataset["Fare"].isnull().sum()
display(dataset[num_cols].isnull().sum())

* Check the skewness of Fare feature

In [ ]:
# before logarithm - display Fare Distribution
show_dist_plot(dataset, 'Fare')

As we can see, Fare distribution is very skewed. This can lead to overweigth very high values in the model, even if it is scaled. 
위의 그래프처럼 , Fare band가 매우 편향되어있음을 알수 있다. 이런 데이터를 그냥 모델에 학습시키면 Min/Max Scaler혹은 Standard Scaler로 스케일링이 되어 있다 하더락도 과적합되기 마련이다.

그러므로 이런 편향된 데이터의 경우엔, 로그변환을 적용해야 한다.

np.log1p변환과 np.log()변환을 같이 한다면 이때의 편향정도는?
What if we do both np.log1p and np.log ?

* Logarithm transformation

In [ ]:
dataset["Fare"] = dataset["Fare"].apply(lambda x:np.log1p(x))

# The following method is also for logarithm
#dataset["Fare"] = dataset["Fare"].map(lambda i: np.log(i) if i > 0 else 0)

In [ ]:
# After Logarithm
show_dist_plot(dataset, 'Fare')

로그변환 후에 편향정도가 상당히 감소했음을 알 수 있다

### 3.2 범주형 피처의 값

In [ ]:
def show_factor_plot(df, xCol=None, yCol=None, col=None, kind='bar', hue=None):
    """
    피처별 생존률 비교
    """
    g = sns.factorplot(x=xCol, y=yCol, hue=hue, col=col, kind=kind, palette="muted", size=6, data = df)
    sns.despine(left=True)
    g.set_ylabels(yCol)

#### Sex
* bar plot으로 성별 생존률
* Sex로 Groupby하여 Survived의 mean()값 

In [ ]:
g = sns.barplot(x="Sex", y = "Survived", data=train)
g = g.set_ylabel("Survival Probability for Sex")

In [ ]:
train[["Sex", "Survived"]].groupby("Sex").mean()

사고가 났을때 100명중에 20명정도는 남자는 살고, 여자는 75명 이상은 산다고 나타난다

성별 피처는 생존률을 예측할때 정말 중요한 피처라고 할 수 있다

1997년 타이타닉 영화가 나왔을때 영화대사 중에 이런 구절이 나온다.
: "여자와 아이들 먼저 구조해". 

세월호 침몰 사고때는 어떠했는가?  

#### Pclass

In [ ]:
# Explore Pclass vs Survived
show_factor_plot(train, "Pclass", 'Survived')

In [ ]:
# Explore Pclass vs Survived by Sex
show_factor_plot(train, "Pclass",'Survived', hue='Sex')

* 좌석별 여성 및 남성 비교

In [ ]:
dataset[(dataset["Pclass"]==3)].groupby('Sex')['Pclass'].count()

승객들의 생존률은 3등급 객실이라고 해서 다르지 않다. 1등급 객실은 2등급, 3등급 객실보다는 생존률이 높다.

#### Embarked

In [ ]:
dataset["Embarked"].isnull().sum()

In [ ]:
# 사우스햄튼이 가장 승객이 많다.
dataset["Embarked"].value_counts()

In [ ]:
#Fill Embarked nan values of dataset set with 'S' most frequent value
dataset["Embarked"] = dataset["Embarked"].fillna('S')

In [ ]:
# Recheck
dataset["Embarked"].isnull().sum()

In [ ]:
# Explore Embarked vs Survived 
show_factor_plot(train, xCol='Embarked', yCol='Survived')

프랑스 노르망디 해안에 위치한 체르보르그항에서 탑승한 승객들의 생존률이 다른 사우스햄튼 혹은 퀸즈타운보다 훨씬 생존률이 좋음을 알 수 있다.

그렇다면, 1등급 객실에 묵은 승객들의 비율이 다른 경유지 항구 - 체르보르그, 사우스햄튼, 퀸즈타운에서 온 승객들보다 더 많을 것이다.

경유지항구(Embarked) v.s Pclass(객실등급)

In [ ]:
# Explore Pclass vs Embarked 
show_factor_plot(train, xCol='Pclass', col='Embarked', kind='count')

사우스햄튼과 퀸즈타운의 승객들의 대부분은 3등급 객실에 분포되어 있으며, 추론은 이 사망자의 대부분은 퀸즈타운과 사우스햄튼 탑승객 일 것이다.

반면 체르보르그 탑승객들은  1등급이 가장 많다. 

1등급 객실의 생존률이 높은 이유는?
-> 사회적 영향도 인가, 아니면 구조가 용이한 여객선의 구조적 문제인가?

## 4. 결측치 채우기
### 4.1 Age

앞에서 보듯히, Age피처는 256개의 결측치를 포함하고있다.

Age가 속한 연령대에 따라 생존률이 극명하게 갈리는 부분이 있기 때문에 중요한 피처라고 할 수 있으며, 이를 위해서라도 결측치를 대치할 부분이 필요하다.

이를 위해서는 Sex, Parch, Pclass 그리고 SibSp를 이용해 새로운 피처를 생성하는 문제를 생각해보자

In [ ]:
# Explore Age vs Sex, Parch , Pclass and SibSP
g = sns.factorplot(y="Age",x="Sex",data=dataset,kind="box")
g = sns.factorplot(y="Age",x="Sex",hue="Pclass", data=dataset,kind="box")
g = sns.factorplot(y="Age",x="Parch", data=dataset,kind="box")
g = sns.factorplot(y="Age",x="SibSp", data=dataset,kind="box")

성별이 연령대를 예측할 수 있는 결정인자는 아니다.

그러나 1등급 객실에 있는 승객의 나이가 다른 2등급 객실, 3등급 객실의 승객들보다 연령대가 높다는 점은 주목 할 만하다.

부모 및 아이들을 동반한 경우가 자식,배우자를 동반한 경우보다 연령대가 높다는 것을 알 수 있다.

In [ ]:
dataset["Sex"] = dataset["Sex"].apply(lambda x: 0 if x =="male" else 1)
# convert Sex into categoricl value 0 for male and 1 for female
# dataset["Sex"] = dataset["Sex"].map({"male":0, "female":1})

In [ ]:
# 앞에서 한번 np.log1p()로 로그변환 한 데이터를 다시 밑에것 np.logp()를 수행하면 
# 히트맵에서 이상하게 보임
df = dataset[["Sex", "Age", "SibSp", "Parch", "Pclass"]]
g = sns.heatmap(df.corr(), cmap="BrBG", annot=True)

상관관계 맵은 Parch 피처를 제외한 factorplot관측치를 확인합니다. 
Age피처가 Sex랑은 관련성이 떨어지고, Pclass, Parch 그리고 SibSp와 음의 상관관계를 가지고있습니다.

Age피처가 부모 및 아이들의 수에 따라 증가하는 경향은 있지만, 전반적으로는 음의 상관관계입니다.

그래서 Age 결측치를 대치하기 위해 SibSp, Parch 그리고 Pclass를 사용하기로 합니다.(결측치 대치는 median)

In [ ]:
def fill_missing_value_age(df, idx_nan_age: list):
    """
    """
    for i in idx_nan_age:
        age_med = df["Age"].median() 
        age_pred = df["Age"][((df["SibSp"]==df.iloc[i]["SibSp"]) & (df["Parch"]==df.iloc[i]["Parch"]) & (df["Pclass"] == df.iloc[i]["Pclass"]))].median()
        
        if not np.isnan(age_pred):
            df["Age"].iloc[i] = age_pred
        else:
            df["Age"].iloc[i] = age_med
    return df

# Age피처 널인것들을 median값으로 대치
index_NaN_age = list(dataset["Age"].isnull().index)
dataset = fill_missing_value_age(dataset, index_NaN_age)

* [box v.s violin] (https://junklee.tistory.com/9)

In [ ]:
show_factor_plot(train, xCol='Survived', yCol='Age', kind='box')
show_factor_plot(train, xCol='Survived', yCol='Age', kind='violin')

사망자와 생존자의 Age의 median값도 크게 차이가 없다.

violin plot을 보면 나이가 어린 영역대의 생존률이 상당히 높음을 알 수 있다.

## 5. 피처 엔지니어링

In [ ]:
def feature_engineering(df, titleOpt=False, familyOpt=False, oneHotEncode=False, ticketT=False, dropFeature=False):
    """
    1.Title에 대한 정제작업
    2.FamilySize 피처 추가 도출 
    3.원핫인코딩  - Title, Embarked
    4.Treat Ticket by extracting the ticket prefix. When there is no prefix it returns X. 
    5.불필요한 피처 삭제
    """
    
    if titleOpt:
        # 빈도표를 이용해 해당 피처의 리스트를 구하기 
        titleList = df["Title"].value_counts().index.values.tolist()
        #display(titleList)
        df["Title"].replace(titleList, ['Mr', 'Miss', 'Mrs', 'Master', 'Mr', 'Other', 'Other', 'Other', 'Miss', 'Miss', 'Mr', 'Other', 'Mr', 'Miss', 'Other', 'Other', 'Mrs', 'Mr'], inplace=True)
        df['Title'].replace(['Mr','Mrs','Miss','Master','Other'], [0, 1, 2, 3, 4], inplace=True)
        df['Title'].astype(int)
        
        # Drop Name variable
        df.drop(labels = ['Name'], axis = 1, inplace=True, errors='ignore')
    if familyOpt:
        df['FamilySize'] = df['SibSp'] + df['Parch'] + 1
        
        # 피처 추가 후  기존 컬럼 삭제 
        df.drop(labels = ['SibSp', 'Parch'], axis = 1, inplace=True, errors='ignore')  
        
    if oneHotEncode:
        cols = ["Title", "Embarked"]
        
        for i in range(len(cols)):
            if i == 0:
                df = pd.get_dummies(df, columns = [cols[i]])                
            else:
                df = pd.get_dummies(df, prefix = 'Em', columns = [cols[i]])
    
    if ticketT:        
        Ticket = []
        for i in list(df["Ticket"]):
            if not i.isdigit():
                Ticket.append(i.replace(".", "").replace("/", "").strip().split(' ')[0])  # 앞의 문자만 추출
            else:
                Ticket.append("X")
        df["Ticket"] = Ticket
        display(df["Ticket"].head(n=2))
    
    #불필요한 컬럼 삭제 
    if dropFeature:
        df.drop(labels=['PassengerId'], axis=1, inplace=True, errors='ignore')    
    
    display(df.head(n=2))
    display(df.tail(n=2))
    return df

### 5.1 Name/Title

In [ ]:
display(dataset["Name"].head())
display(dataset["Name"].tail())

The Name feature contains information on passenger's title.

Since some passenger with distingused title may be preferred during the evacuation, it is interesting to add them to the model.

In [ ]:
# Get Title from Name
dataset_title = [i.split(',')[1].split('.')[0].strip() for i in dataset["Name"]]
display(dataset_title[1: 10])
dataset["Title"] = pd.Series(dataset_title)

display(dataset.head(n=2))

In [ ]:
g = sns.countplot(x="Title", data = dataset)
g = plt.setp(g.get_xticklabels(), rotation=45)

There is 17 titles in the dataset, most of them are very rare and we can group them in 4 categories.

In [ ]:
dataset = feature_engineering(dataset, titleOpt=True)

In [ ]:
g = sns.countplot(dataset["Title"])
g = g.set_xticklabels(["Mr","Mrs","Miss", "Master", "Other"])

In [ ]:
g = sns.factorplot(x='Title', y='Survived', data = dataset, kind='bar')
g = g.set_xticklabels(["Mr","Mrs","Miss", "Master", "Other"])
g = g.set_ylabels("Survival probability for Title")

"여자와 아이먼저 구조" 

"Other" 타이틀이 승선객의 Title구성을 볼때 생존률이 높다.

### 5.2 Family size

* 가족수(FamilySize)피처 추가  - SibSp(동반가족(자손 + 배우자)) , Parch(부모님 동반된 아이들 포함)

In [ ]:
# Create a family size descriptor from SibSp and Parch
dataset = feature_engineering(dataset, familyOpt=True)

In [ ]:
g = sns.factorplot(x="FamilySize", y = "Survived", data = dataset)
g = g.set_ylabels("Survival Probability for FamilySize")

The family size seems to play an important role, survival probability is worst for large families.

Additionally, i decided to created 4 categories of family size.

In [ ]:
dataset["FamilySize"].value_counts()

In [ ]:
dataset["Single"] = dataset["FamilySize"].map(lambda s: 1 if s ==1 else 0)
dataset["SmallF"] = dataset["FamilySize"].map(lambda s: 1 if s ==2 else 0)
dataset["MedF"] = dataset["FamilySize"].map(lambda s: 1 if 3 <= s <= 4 else 0)
dataset["LargeF"] = dataset["FamilySize"].map(lambda s: 1 if s >= 5 else 0)

In [ ]:
def show_factorplot(df, cols):
    """
    데이터셋과 컬럼에 대한 정보를 받아서 
    Factotplot으로 피처별 생존률을 보여줌
    """
    for i in range(len(cols)):
        g = sns.factorplot(x=cols[i], y='Survived', data=dataset, kind='bar')
        g = g.set_ylabels("Survival Probabilit for {0}".format(cols[i]))   
    
# 피처별 생존률    
cols = ["Single" ,"SmallF", "MedF", "LargeF"]
show_factorplot(dataset, cols)

2인, 3인, 4인정도의 인원으로 가족끼리 탑승한 경우 생존률이 1인 혹은 5인 이상의 가족 및 친지끼리 승선한 경우보다 생존률이 좋다는 점은 기억하자. 

혼자보다 둘이, 둘보다는 세명이서 여행가라. 그러면 사고당해서 돌아오지 못할 확률이 줄어든다.

In [ ]:
dataset.info()

In [ ]:
# One-Hot Encoding Title and Embarked 
#dataset = feature_engineering(dataset, oneHotEncode=True)
dataset = pd.get_dummies(dataset, columns=["Title"])
dataset = pd.get_dummies(dataset, columns=["Embarked"], prefix="Em")

In [ ]:
display(dataset.head())
display(dataset.info())

At this stage, we have 22 features.

### 5.3 Cabin

In [ ]:
def display_feature_info(df, feature):
    """
    """
    display(df[feature].head())
    print("\n")
    display(df[feature].describe())
    
    print("\n")
    # 널값 갯수
    display(dataset[feature].isnull().sum())
    
    print("\n")
    display(dataset[feature][dataset[feature].notnull()].head())
    display(dataset[feature][dataset[feature].notnull()].tail())

In [ ]:
display_feature_info(dataset, 'Cabin')

Cabin피처는 292개는 값이 존재하고 나머지는 1007개는 널값으로 존재

값이 없는 피처의 값은 'X'로 하고 있는 피처의 값은 앞의 첫글자만 추출해서 보여준다.

In [ ]:
dataset["Cabin"] = [i[0] if not pd.isnull(i) else 'X' for i in dataset["Cabin"]]

In [ ]:
cabin_sort = dataset["Cabin"].value_counts().index.values
sorted(cabin_sort)

The first letter of the cabin indicates the Desk, i choosed to keep this information only, since it indicates the probable location of the passenger in the Titanic.

In [ ]:
g = sns.countplot(dataset["Cabin"], order = sorted(cabin_sort))

In [ ]:
g = sns.factorplot(y='Survived', x='Cabin', data=dataset, kind='bar', order=sorted(cabin_sort))
g = g.set_ylabels("Survival Probability for Cabin")

Cabin피처의 경우 널값을 대체하기전 292개의 값만 널이 아니었기 때문에 크게 의미가 있다고 할순없지만, 값이 있는 피처들의 기준으로만 보면 B, C, D, E, F값을 갖는 Cabin의 경우는 생존률이 높음에 주목하라.

대체적으로 Cabin의 값이 널이었던 'X'의 경우는 생존률이 많이 낮음을 알수있다. 

* One-Hot Encoding

In [ ]:
dataset = pd.get_dummies(dataset, columns=["Cabin"], prefix = 'Cabin')

In [ ]:
display(dataset.head(n=2))
display(dataset.info())

### 5.4 Ticket

In [ ]:
Ticket = []
for i in list(dataset["Ticket"]):
    if not i.isdigit():
        Ticket.append(i.replace(".", "").replace("/", "").strip().split(' ')[0])  # 앞의 문자만 추출
    else:
        Ticket.append("X")
        
display(Ticket[1:10])        

In [ ]:
display_feature_info(dataset, 'Ticket')

Ticket은 피처의 성격상, 객실의 등급이 티겟번호에 숨어있는 의미가 있기때문에, 우리는 여기서 좀더 있는 의미있는 결과를 위해 Ticket피처의 Prefix를 추출해서 관리하기로 함

In [ ]:
## Treat Ticket by extracting the ticket prefix. When there is no prefix it returns X. 
dataset = feature_engineering(dataset, ticketT=True)

* One-Hot Encoding

In [ ]:
dataset = pd.get_dummies(dataset, columns=["Ticket"], prefix='T')

In [ ]:
display(dataset.head(n=2))
display(dataset.info())

In [ ]:
dataset["Pclass"] = dataset["Pclass"].astype('category')
dataset = pd.get_dummies(dataset, columns=['Pclass'], prefix='Pc')

In [ ]:
# Drop useless variables 
feature_engineering(dataset, dropFeature=True)

In [ ]:
dataset.info()

## 6. MODELING

In [ ]:
train = dataset[:train.shape[0]]
test = dataset[train.shape[0]:]
test.drop(labels=["Survived"],axis = 1,inplace=True)

In [ ]:
## Separate train features and label 

train["Survived"] = train["Survived"].astype(int)

Y_train = train["Survived"]

X_train = train.drop(labels = ["Survived"],axis = 1)

display(X_train.shape, Y_train.shape)

### 6.1 단순 모델링
#### 6.1.1 교차검증 모델 

10개의 인기있는 분류기를 통해 각각 Stratified KFold교차검증을 이용해 평균 정확도를 평가할 것이다.

* SVC
* Decision Tree
* AdaBoost 
* Random Forest
* Extra Trees
* Gradient Boosting
* Multiple layer perceprton (neural network)
* KNN
* Logistic regression
* Linear Discriminant Analysis

#### **모델 평가**

In [ ]:
# Cross validate model with Kfold stratified cross val
kfold = StratifiedKFold(n_splits = 10)

In [ ]:
def model_evaluation(classifiers, X, y, kfold):
    """
    모델에 대한 예측 정확도 평가 
    """
    cv_results = []
    classifier_name = []
    
    for classifier in classifiers:
        cv_results.append(cross_val_score(classifier, X, y, scoring='accuracy', cv = kfold, n_jobs = 4,verbose=True))
        classifier_name.append(classifier.__class__.__name__)
        
    cv_means = []
    cv_std = []
    
    for cv_result in cv_results:
        cv_means.append(cv_result.mean())
        cv_std.append(cv_result.std())
        
    # 데이터프레임으로 리턴
    cv_res = pd.DataFrame(data = {'CrossValMeans': cv_means, "CrossValerrors": cv_std, "Algorithm": classifier_name})
    
    return cv_res, cv_std, cv_means

#### **수행결과**

#### **모델 평가 수행**

In [ ]:
random_state = 2
svc = SVC(random_state=random_state)
dt_clf = DecisionTreeClassifier(random_state = random_state)
ada_clf = AdaBoostClassifier(DecisionTreeClassifier(random_state=random_state), n_estimators=100, learning_rate=0.05,random_state=random_state)
rf_clf  = RandomForestClassifier(n_estimators = 150, n_jobs = -1, random_state=random_state)
ext_clf = ExtraTreesClassifier(random_state=random_state)
grd_clf = GradientBoostingClassifier(learning_rate = 0.5, n_estimators = 500, random_state=random_state)
mlp_clf = MLPClassifier(random_state=random_state)
knn_clf = KNeighborsClassifier(n_neighbors = 10)
lr_clf  = LogisticRegression(random_state = random_state)
lda_clf = LinearDiscriminantAnalysis()

classifiers = []
classifiers.append(svc)
classifiers.append(dt_clf)
classifiers.append(ada_clf)
classifiers.append(rf_clf)
classifiers.append(ext_clf)
classifiers.append(grd_clf)
classifiers.append(mlp_clf)
classifiers.append(knn_clf)
classifiers.append(lr_clf)
classifiers.append(lda_clf)

# 정확도 평가
cv_res, cv_std, cv_means = model_evaluation(classifiers, X_train, Y_train, kfold)

In [ ]:
cv_res, cv_std, cv_means

#### **모델별로 정확도 시각화**

In [ ]:
g = sns.barplot(x="CrossValMeans", y="Algorithm", data=cv_res, palette="Set3",orient = "h",**{'xerr':cv_std})
g.set_xlabel("Mean Accuracy")
g = g.set_title("Cross Validation Scores")

앙상블 모델링으로 GradientBoostingClassifier, ExtraTrees, RandomForest, AdaBoost, SVC를 사용할 것임.

#### 6.1.2 최적의 모델을 위한 하이퍼 파라미터 튜닝

GradientBoostingClassifier, ExtraTrees, RandomForest, AdaBoost, SVC를 GridSearchCV를 이용해 하이퍼 파라미터 튜닝

In [ ]:
a = {'국어': [20, 30, 40],'영어':[10, 20, 30], '수학': [20, 56, 90]}

pd.DataFrame(data = a)

* 최적의 ML모델을 가져오기 위한 함수

In [ ]:
def get_best_estimator(models, X_train, Y_train):
    """
    각 모델에 대한 최적의 하이퍼 파라미터를 튜닝
    """
    best_model = []
    best_params = []
    best_score = []
    
    for model in models:
        model.fit(X_train, Y_train)
        
        display("Best Estimator : ", model.best_estimator_)
        display("Best Params : ", model.best_params_)
        display("Best Score : ", model.best_score_)
        
        best_model.append(model.best_estimator_)
        best_params.append(model.best_params_)
        best_score.append(model.best_score_)
        
    result = pd.DataFrame({
                    "Best Model": best_model,
                    "Best Params": best_params,
                    "Best Score":best_score
                 })
    display("모델의 예측 성능 : \n", result)
    return result

* ML모델별 최적화 파라미터 설정

In [ ]:
# AdaBoost
models = []
dt_clf  = DecisionTreeClassifier()
adaDTC = AdaBoostClassifier(dt_clf, random_state=7)

ada_param_grid = {
    'n_estimators': [1, 2, 10, 50],
    'algorithm': ['SAMME', 'SAMME.R'],
    'learning_rate': [0.0001, 0.001, 0.01, 0.1, 0.2, 0.3,1.5]
}
gsadaDTC = GridSearchCV(estimator=adaDTC, param_grid=ada_param_grid, scoring='accuracy', n_jobs=-1, verbose=True, cv=kfold)

models.append(gsadaDTC)
# ExtraTrees
Extc = ExtraTreesClassifier()

ex_param_grid = {
                "criterion": ["gini", "entropy"],
                "max_depth": [None],
                "max_features": [1, 3, 10],
                "min_samples_split": [2, 3, 10],
                "min_samples_leaf": [1, 3, 10],
                "bootstrap": [False],
                "n_estimators" :[100,300]                
}

gsExtc = GridSearchCV(Extc, param_grid=ex_param_grid, scoring='accuracy', n_jobs=-1, verbose=True, cv=kfold)
models.append(gsExtc)

# RandomForestClassifier
rf_clf = RandomForestClassifier()

rf_param_grid = {"max_depth": [None],
              "max_features": [1, 3, 10],
              "min_samples_split": [2, 3, 10],
              "min_samples_leaf": [1, 3, 10],
              "bootstrap": [False],
              "n_estimators" :[100,300],
              "criterion": ["gini"]}

gsRfc = GridSearchCV(rf_clf, param_grid=rf_param_grid, scoring='accuracy', n_jobs= -1, verbose=True, cv=kfold)
models.append(gsRfc)


# GradientBoostClassifier
gdb_clf = GradientBoostingClassifier()

gdb_param_grid = {
                'loss': ['deviance', 'exponential'],
                'n_estimators' : [100,200,300],
                'learning_rate': [0.1, 0.05, 0.01],
                'max_depth': [4, 8],
                'min_samples_leaf': [100,150],
                'max_features': [0.3, 0.1]                 
}

gsGdb = GridSearchCV(gdb_clf, gdb_param_grid, scoring='accuracy', n_jobs = -1, cv=kfold)
models.append(gsGdb)

#SVM
svcs = SVC(probability = True)

svc_param_grid = {
    'kernel': ['rbf'],
    'C': [1, 10, 50, 100,200,300, 1000],
    'gamma': [0.001, 0.01, 0.1, 1]
}

gsSvc = GridSearchCV(svcs, param_grid = svc_param_grid, scoring='accuracy', n_jobs = -1, cv = kfold)
models.append(gsSvc)

In [ ]:
result = get_best_estimator(models, X_train, Y_train)

#### 6.1.3 Plot learning curves

학습곡선은 예측 정확도에 대한 학습크기의 효과 및 학습세트에 대한 과적합  결과를 보기위한 좋은 방법입니다.

* [Learning Curve](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.learning_curve.html)
* [Data playground](https://dataplay.tistory.com/32?category=855225)

In [ ]:
def plot_learning_curve(estimator, title, X, y, ylim=None, cv=None,
                        n_jobs=-1, train_sizes=np.linspace(.1, 1.0, 5)):
    """Generate a simple plot of the test and training learning curve"""
    plt.figure()
    plt.title(title)
    if ylim is not None:
        plt.ylim(*ylim)
    plt.xlabel("Training examples")
    plt.ylabel("Score")
    train_sizes, train_scores, test_scores = learning_curve(
        estimator, X, y, cv=cv, n_jobs=n_jobs, train_sizes=train_sizes)
    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)
    plt.grid()

    plt.fill_between(train_sizes, train_scores_mean - train_scores_std,
                     train_scores_mean + train_scores_std, alpha=0.1,
                     color="r")
    plt.fill_between(train_sizes, test_scores_mean - test_scores_std,
                     test_scores_mean + test_scores_std, alpha=0.1, color="g")
    plt.plot(train_sizes, train_scores_mean, 'o-', color="r",
             label="Training score")
    plt.plot(train_sizes, test_scores_mean, 'o-', color="g",
             label="Cross-validation score")

    plt.legend(loc="best")
    return plt

In [ ]:
for best_model in result["Best Model"]:
    g = plot_learning_curve(best_model, best_model.__class__.__name__ + " learning curves", X_train, Y_train, cv=kfold)

GradientBoosting및 Adaboost 분류기가 학습세트에 과적합의 경향이 보인다.
교차검증횟수가 증가함에 따라, GradientBoosting과 Adaboost가 학습데이터의 크기에 따라 성능이 좋아짐을 보인다.

SVC와 ExtraTrees는 학습점수와 교차검증 점수가 서로 수렴하는 양상을 보이고있다. 

#### 6.1.4 Feature importance of tree based classifiers

승객의 생존률을 예측하기 위한 가장 중요한 피처는 무엇인지 feature_importance를 통해 알아본다.

In [ ]:
nrows = ncols = 2
fig, axes = plt.subplots(nrows = nrows, ncols = ncols, sharex='all', figsize=(15, 15))

names_classifiers = [(best_model.__class__.__name__, best_model) for best_model in result["Best Model"]]

nclassifier = 0
for row in range(nrows):
    for col in range(ncols):
        name = names_classifiers[nclassifier][0]
        classifier = names_classifiers[nclassifier][1]
        
        indices = np.argsort(classifier.feature_importances_[::-1][:40])
        g = sns.barplot(y = X_train.columns[:40], x = classifier.feature_importances_[:40], orient='h', ax = axes[row, col])
        g.set_xlabel("Relative Importances", fontsize=12)
        g.set_ylabel("Features", fontsize=12)
        g.tick_params(labelsize=9)
        g.set_title(name + " feature importances")     
        
        nclassifier += 1

**Age, Sex 그리고 FamilySize가 생존률 예측에 중요 인자**

In [ ]:
predict_series = [pd.Series(best_model.predict(test), name=best_model.__class__.__name__) for best_model in result["Best Model"]] 
ensemble_results = pd.concat(predict_series, axis = 1)

g = sns.heatmap(ensemble_results.corr(), annot=True)

The prediction seems to be quite similar for the 5 classifiers except when Adaboost is compared to the others classifiers.

The 5 classifiers give more or less the same prediction but there is some differences. Theses differences between the 5 classifier predictions are sufficient to consider an ensembling vote. 

### 6.2 앙상블 모델 
#### 6.2.1 ML모델 조합

5개의 ML모델을 사용해 예측한 평가를 앙상블모델을 이용해 soft voting예측(각 모델의 예측의 평균치를 합산)

In [ ]:
votingC = VotingClassifier(estimators= names_classifiers)

votingC.fit(X_train, Y_train)

### 6.3 예측
#### 6.3.1 예측 및 결과 제출

In [ ]:
test_survived = pd.Series(votingC.predict(test), name="Survived")
result = pd.concat([IDTEST, test_survived], axis = 1)

result.to_csv("ensemble_survival_prediction.csv", index=False)